In [ ]:
import os      
import pandas as pd
import numpy as np
import requests
import re

from lxml import etree
from urllib.request import urlretrieve
import time
from tqdm import tqdm
from urllib.request import urlopen
from urllib.request import urlretrieve
import cgi

In [ ]:
def get_url(str):            
    return str.split('"')[1]

In [ ]:
def pdf_url(url):
    html = requests.get(url).text
    url = re.findall("(?<=href=).*(?=class)", html) 
    return "http://news.windin.com/ns/" + url[0] 

In [ ]:
def get_pdf_url(df):
    try:
        with tqdm (range(0, df.shape[0]), ncols=80, desc = "爬取文件链接 进度") as t:
            for i in t:
                times = 1
                while times <= 3:
                    try:
                        df["PDF地址"][i]= pdf_url(df["公告地址"][i])
                        break
                    except OSError:
                        times += 1
                        print("获取 ", df["公告标题"][i], " 链接时出现问题")
                        print(f"休息5秒！再试第{times}次!")
                        time.sleep(5)
    except KeyboardInterrupt:
        t.close()
        raise
    t.close()

In [ ]:
def change_name(df):
    for i in range (0, df.shape[0]):
        df.iloc[i][2] = df.iloc[i][2].replace(":", "")
        df.iloc[i][2] = df.iloc[i][2].replace("*", "")

In [ ]:
def download(download_path, df):
    try:
        with tqdm (range(0,df.shape[0]),desc = "文件下载 进度", ncols=80) as t:
            for i in t:
                times = 1
                while times <= 3:
                    try:
                        name = df["证券代码"][i][:6] + "_" + df['公告标题'][i] + "_" + df["公告日期"][i] #文件名称
                        url = df["PDF地址"][i] #文件地址
                        remotefile = urlopen(url)
                        blah = remotefile.info()['Content-Disposition']
                        value, params = cgi.parse_header(blah)
                        filename = name + "." + params["filename"].split(".")[-1] #获取扩展名                  
                        urlretrieve(url, filename =  download_path + filename) #下载p文件
                        break
                    except OSError:
                        times += 1
                        print("下载", df["公告标题"][i], "时出现问题")
                        print(f"休息5秒！再试第{times}次!")
                        time.sleep(5)
                
                #print(f"成功下载{name}！")
            print("成功下载所有文件！")
    
    except KeyboardInterrupt:
        t.close()
        raise
    t.close()

In [ ]:
def run(download_path, df):
    df["公告地址"] = df["公告地址"].apply(get_url)
    df["PDF地址"] = np.nan
    get_pdf_url(df)
    change_name(df)
    return df

In [ ]:
def main(excel_path, download_path):
    df = pd.read_excel(excel_path)
    df = run(download_path, df)
    download(download_path, df)
    return df

In [ ]:
if __name__ == '__main__':
    excel_path = "E:\\NLP_Dataset\\会议记录\\xls\\公司公告2020-06-27(26).xlsx"
    download_path = "E:\\NLP_Dataset\\活动记录表\\非银行金融\\pdf\\" #以\\结尾
    df = main (excel_path, download_path)